In [12]:
import pandas as pd
from sqlalchemy import create_engine
from config import password
from sqlalchemy.sql import text as sa_text

rds_connection_string = f'postgres:{password}@localhost:5432/thor'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
# display the table names in the thor database
engine.table_names()

['thor_wwii_weather',
 'thor_wwii_station_location',
 'thor_wwii_aircraft',
 'thor_wwii_master']

In [15]:
# truncate the tables to get rid of existing data
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_master CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_aircraft CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_station_location CASCADE;''').execution_options(autocommit=True))
engine.execute(sa_text('''TRUNCATE TABLE thor_wwii_weather CASCADE;''').execution_options(autocommit=True))

## Step 1. thor_wwii_aircraft ETL

In [16]:
# path to the thor_wwii_aircraft file
aircraft_csv = './Resources/THOR_WWII_AIRCRAFT_GLOSS.csv'


# load the THOR_WWII_AIRCRAFT_GLOSS.csv file into a df
aircraft_df = pd.read_csv(aircraft_csv)
aircraft_df.head()

,GLOSS_ID,AIRCRAFT,NAME,FULL_NAME,AIRCRAFT_TYPE,HYPERLINK
0,1,A20,A20,Douglas A-20 Havoc,Boston Light Bomber / Night-Fighter,http://militaryfactory.com/aircraft/detail.asp...
1,2,A24,A24,Douglass A-24 Banshee,Dive Bomber / Reconnaissance,http://militaryfactory.com/aircraft/detail.asp...
2,3,A26,A26,Douglas A-26 Invader,Medium Bomber / Heavy Assault,http://militaryfactory.com/aircraft/detail.asp...
3,4,A36,A36,North American A-36 Apache (Invader),Ground Attack / Dive Bomber,http://militaryfactory.com/aircraft/detail.asp...
4,5,ALBA,Albacore,Fairey Albacore,Naval Torpedo Bomber,http://militaryfactory.com/aircraft/detail.asp...


In [17]:
# Create a new dataframe with the fields needed for the insert
new_aircraft_df = aircraft_df[['AIRCRAFT', 'NAME', 'FULL_NAME', 'AIRCRAFT_TYPE','HYPERLINK']].copy()
new_aircraft_df = new_aircraft_df.rename(columns={
                                                    'AIRCRAFT': 'aircraft', 
                                                    'NAME': 'name',
                                                    'FULL_NAME': 'full_name',
                                                    'AIRCRAFT_TYPE':'aircraft_type',
                                                    'HYPERLINK':'hyperlink'
})

new_aircraft_df.head()

,aircraft,name,full_name,aircraft_type,hyperlink
0,A20,A20,Douglas A-20 Havoc,Boston Light Bomber / Night-Fighter,http://militaryfactory.com/aircraft/detail.asp...
1,A24,A24,Douglass A-24 Banshee,Dive Bomber / Reconnaissance,http://militaryfactory.com/aircraft/detail.asp...
2,A26,A26,Douglas A-26 Invader,Medium Bomber / Heavy Assault,http://militaryfactory.com/aircraft/detail.asp...
3,A36,A36,North American A-36 Apache (Invader),Ground Attack / Dive Bomber,http://militaryfactory.com/aircraft/detail.asp...
4,ALBA,Albacore,Fairey Albacore,Naval Torpedo Bomber,http://militaryfactory.com/aircraft/detail.asp...


In [18]:
# load the data into the table
new_aircraft_df.to_sql(name='thor_wwii_aircraft', con=engine, if_exists='append', index=False)

In [19]:
# test that the data was loaded into the database
pd.read_sql_query('select * from thor_wwii_aircraft', con=engine).head()

,aircraft,name,full_name,aircraft_type,hyperlink
0,A20,A20,Douglas A-20 Havoc,Boston Light Bomber / Night-Fighter,http://militaryfactory.com/aircraft/detail.asp...
1,A24,A24,Douglass A-24 Banshee,Dive Bomber / Reconnaissance,http://militaryfactory.com/aircraft/detail.asp...
2,A26,A26,Douglas A-26 Invader,Medium Bomber / Heavy Assault,http://militaryfactory.com/aircraft/detail.asp...
3,A36,A36,North American A-36 Apache (Invader),Ground Attack / Dive Bomber,http://militaryfactory.com/aircraft/detail.asp...
4,ALBA,Albacore,Fairey Albacore,Naval Torpedo Bomber,http://militaryfactory.com/aircraft/detail.asp...


## Step 2. thor_wwii_station_location ETL

In [20]:
# path to the thor_wwii_station_location file
station_csv = './Resources/Weather Station Locations.csv'

# load the Weather Station Locations.csv file into a df
station_df = pd.read_csv(station_csv)
station_df.head()

,WBAN,NAME,STATE/COUNTRY ID,LAT,LON,ELEV,Latitude,Longitude
0,33013,AIN EL,AL,3623N,00637E,611,36.383333,6.650000
1,33031,LA SENIA,AL,3537N,00037E,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,3643N,00314E,23,36.716667,3.216667
3,33044,TELERGMA,AL,3607N,00621E,754,36.116667,6.416667
4,12001,TINDOUF,AL,2741N,00809W,443,27.683333,-8.083333


In [21]:
# Create a new dataframe with the fields needed for the insert
new_station_df = station_df[['WBAN', 'NAME', 'STATE/COUNTRY ID', 'ELEV','Latitude','Longitude']].copy()
new_station_df = new_station_df.rename(columns={
                                                    'WBAN': 'wban', 
                                                    'NAME': 'name',
                                                    'STATE/COUNTRY ID': 'state_country',
                                                    'ELEV':'elev',
                                                    'Latitude':'latitude',
                                                    'Longitude': 'longitude'
                            
})

new_station_df.head()

,wban,name,state_country,elev,latitude,longitude
0,33013,AIN EL,AL,611,36.383333,6.650000
1,33031,LA SENIA,AL,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,23,36.716667,3.216667
3,33044,TELERGMA,AL,754,36.116667,6.416667
4,12001,TINDOUF,AL,443,27.683333,-8.083333


In [22]:
# load the data into the table
new_station_df.to_sql(name='thor_wwii_station_location', con=engine, if_exists='append', index=False)

In [23]:
# test that the data was loaded into the database
pd.read_sql_query('select * from thor_wwii_station_location', con=engine).head()

,wban,name,state_country,elev,latitude,longitude
0,33013,AIN EL,AL,611,36.383333,6.650000
1,33031,LA SENIA,AL,88,35.616667,0.583333
2,33023,MAISON BLANCHE,AL,23,36.716667,3.216667
3,33044,TELERGMA,AL,754,36.116667,6.416667
4,12001,TINDOUF,AL,443,27.683333,-8.083333


## Step 3. thor_wwii_weather ETL

In [24]:
# path to the thor_wwii_weather file
weather_csv = './Resources/Summary of Weather.csv'

# load the Weather Station Locations.csv file into a df
weather_df = pd.read_csv(weather_csv,low_memory=False)
weather_df.head()

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Create a new dataframe with the fields needed for the insert
new_weather_df = weather_df[['STA', 'Date', 'PRCP', 'DR','SPD','MAX','MIN','MEA','SNF']].copy()
new_weather_df = new_weather_df.rename(columns={
                                                    'STA': 'sta',
                                                    'Date': 'date',
                                                    'PRCP': 'prcp',
                                                    'DR': 'dr',
                                                    'SPD':'spd',
                                                    'MAX':'max',
                                                    'MIN': 'min',
                                                    'MEA': 'mea',
                                                    'SNF': 'snf'
                            
})

new_weather_df.head()

,sta,date,prcp,dr,spd,max,min,mea,snf
0,10001,1942-7-1,0.04,NaN,NaN,78.0,72.0,75.0,0
1,10001,1942-7-2,0,NaN,NaN,84.0,71.0,78.0,0
2,10001,1942-7-3,0.1,NaN,NaN,79.0,72.0,76.0,0
3,10001,1942-7-4,0.1,NaN,NaN,80.0,72.0,76.0,0
4,10001,1942-7-5,0,NaN,NaN,80.0,71.0,76.0,0


In [ ]:
# load the data into the table
new_weather_df.to_sql(name='thor_wwii_weather', con=engine, if_exists='append', index=False)

In [ ]:
# test that the data was loaded into the database
pd.read_sql_query('select * from thor_wwii_station_location', con=engine).head()

### Step 4. thor_wwii_master ETL

In [ ]:
# path to the thor_wwii_weather file
thor_csv = './Resources/THOR_WWII_DATA_CLEAN.csv'

# load the Weather Station Locations.csv file into a df
thor_df = pd.read_csv(thor_csv, encoding = "ISO-8859-1", low_memory=False)
thor_df.head()